## Applied data Science project

#### Importing Libraries & using BeautifulSoup to extract the table from Wiki page

In [121]:
import urllib.request
import pandas as pd

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
req = urllib.request.urlopen(url)
article = req.read().decode()

with open('List_of_postal_codes_of_Canada:_M.html', 'w') as fo:
    fo.write(article)
    
from bs4 import BeautifulSoup

# Load article, turn into soup and get the <table>s.
article = open('List_of_postal_codes_of_Canada:_M.html').read()
soup = BeautifulSoup(article, 'html.parser')
tables = soup.find_all('table', class_='sortable')

# Search through the tables for the one with the headings we want.
for table in tables:
    ths = table.find_all('th')
    headings = [th.text.strip() for th in ths]
    if headings[:3] == ['Postcode', 'Borough', 'Neighbourhood']:
        break
# Extract the columns we want and write to a semicolon-delimited text file.

with open('List_of_postal_codes_of_Canada:_M.txt', 'w') as fo:
    for tr in table.find_all('tr'):
        tds = tr.find_all('td')
        
        if not tds:
            continue
        Postcode, Borough, Neighbourhood = [td.text.strip() for td in tds[:4]]
        # Wikipedia does something funny with country names containing
        # accented characters: extract the correct string form.
        #if '!' in country:
         #   country = country[country.index('!')+1:]
        print('; '.join([Postcode, Borough, Neighbourhood]), file=fo)
    

df = pd.read_csv('List_of_postal_codes_of_Canada:_M.txt', sep=';', header=None, names=['Postcode', 'Borough', 'Neighbourhood'])


#### Drop the rows with Borough = ' Not assigned', group rows with same Postcode & replace Neighbourhood with value of Borough wherever Not assigned

In [105]:
df.drop(df[df['Borough'] == ' Not assigned'].index, inplace = True, axis = 0)
df1 = df[['Postcode', 'Borough']]
df = df.groupby('Postcode')['Neighbourhood'].agg(','.join).reset_index()
df1.drop_duplicates(keep='first',inplace = True)
data = pd.merge(left=df1,right=df, left_on='Postcode', right_on='Postcode')
data['Neighbourhood'][data[data['Neighbourhood']==' Not assigned'].index]= data['Borough'][data[data['Neighbourhood']==' Not assigned'].index]

In [122]:
data.shape

(103, 3)